# Refinitiv Data Library for Python
## Content - Pricing - Streaming Events

This notebook demonstrates using a Pricing object to receive streaming real-time data via the event driven interface.  

## Import the library and load credentials

Credentials used by this and the other tutorials notebooks are stored in the **Configuration/credentials.ipynb** file.     

You should have edited the **Configuration/credentials.ipynb** to set your credentials as part of the **Quick Start** step.

In [1]:
from refinitiv.data.content import pricing
from pandas import DataFrame
from IPython.display import display, clear_output

%run ../../Configuration/credentials.ipynb

## Open the session of your choice

Use our helper function in the Credentials notebook, **open_session(session_type)**, to create and open a session to connect to the 
- Refinitiv Data Platform directly (session_type="rdp") or via 
- Eikon 4 or Refinitiv Workspace (session_type="desktop") or via a 
- local realtime infrastructure (session_type="deployed").

You can also set a default in the **credentials** notebook - which I have done, and so I don't need to specify a session_type:

In [7]:
# I have defined a default Session type in the Credentials notebook - so no need to provide one here
open_session()

<refinitiv.data.session._platform_session.Definition object at 0x1df34756790 {session_name='default-session'}>

### Define a callback to receive streaming data

In [3]:
# Function to handle the intial Refresh for each item
def handle_refresh(streaming_prices, instrument_name, fields):
    # One way to access data - get dataframe
    print(f"Refresh : {streaming_prices.get_snapshot()}")
    return

# Function to update dataframe, when we receive updates for individual items
def handle_update(streaming_prices, instrument_name, fields):
    # Alternative way of accesing data - access the updated fields
    print(f"Update : {instrument_name}:{fields}")
    
# Function to extract status code for an item as & when received from server
# Status contains a 'code' and a more detailed 'message'
def handle_status(streaming_prices, instrument_name, status):
    print(f"Status : {instrument_name}:{status['code']}:{status['message']}")

### Create streams for our RICs and open them

In [8]:
# Define our Streaming Price object - including a bad ric for illustrative purposes
streams = rd.content.pricing.Definition(
    ['EUR=', 'GBP=', 'JPY=', 'BADRIC'],
    fields=['BID', 'ASK']
).get_stream()

# Callback for if we wanted to handle invidiual Refresh for each item
streams.on_refresh(handle_refresh)
# Specify callback handler for any updates
streams.on_update(handle_update)
# Specify callback handler for any updates
streams.on_status(handle_status)

# Send the requests to the server and open the streams for all items
streams.open()

[2021-09-13 14:03:41,451] - [INFO] - [default-session] - [11028] | WebSocket 0 - OMM Protocol - PRICING
Login to websocket wss://apac-3-t3.streaming-pricing-api.refinitiv.com:443/WebSocket successful
Refresh :   Instrument     BID     ASK
0       EUR=  1.1783  1.1787
1       GBP=    <NA>    <NA>
2       JPY=    <NA>    <NA>
3     BADRIC    <NA>    <NA>
Refresh :   Instrument     BID     ASK
0       EUR=  1.1783  1.1787
1       GBP=  1.3832  1.3833
2       JPY=  110.05  110.08
3     BADRIC    <NA>    <NA>
Refresh :   Instrument     BID     ASK
0       EUR=  1.1783  1.1787
1       GBP=  1.3832  1.3833
2       JPY=  110.05  110.08
3     BADRIC    <NA>    <NA>
Status : BADRIC:NotFound:**The record could not be found


<StreamState.Open: 3>

Update : JPY=:{'BID': 110.06, 'ASK': 110.07}
Update : EUR=:{'BID': 1.1783, 'ASK': 1.1786}
Update : EUR=:{'BID': 1.1783, 'ASK': 1.1786}
Update : JPY=:{'BID': 110.05, 'ASK': 110.08}
Update : GBP=:{'BID': 1.383, 'ASK': 1.3834}
Update : EUR=:{'BID': 1.1783, 'ASK': 1.1787}
Update : JPY=:{'BID': 110.06, 'ASK': 110.07}
Update : JPY=:{'BID': 110.06, 'ASK': 110.07}
Update : GBP=:{'BID': 1.3832, 'ASK': 1.3833}
Update : JPY=:{'BID': 110.06, 'ASK': 110.07}
Update : JPY=:{'BID': 110.05, 'ASK': 110.08}
Update : EUR=:{'BID': 1.1784, 'ASK': 1.1785}
Update : GBP=:{'BID': 1.3832, 'ASK': 1.3833}
Update : GBP=:{'BID': 1.3832, 'ASK': 1.3833}
Update : EUR=:{'BID': 1.1783, 'ASK': 1.1787}
Update : GBP=:{'BID': 1.383, 'ASK': 1.3834}


## Close the stream

In [9]:
streams.close()

Update : EUR=:{'BID': 1.1784, 'ASK': 1.1787}


<StreamState.Closed: 1>

## Close the session

In [10]:
close_session()